<a href="https://colab.research.google.com/github/Landa-sivakumar/Ml-project-/blob/main/kiet_team_6_csc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import pandas as pd
import numpy as np
import re  # For extracting numbers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load dataset
df = pd.read_csv("/content/train.csv")

df.head()


,Unnamed: 0,Name,Rating,Spec_score,No_of_sim,Ram,Battery,Display,Camera,External_Memory,Android_version,Price,company,Inbuilt_memory,fast_charging,Screen_resolution,Processor,Processor_name
0,0,Samsung Galaxy F14 5G,4.65,68,"Dual Sim, 3G, 4G, 5G, VoLTE,",4 GB RAM,6000 mAh Battery,6.6 inches,50 MP + 2 MP Dual Rear &amp; 13 MP Front Camera,"Memory Card Supported, upto 1 TB",13,"9,999",Samsung,128 GB inbuilt,25W Fast Charging,2408 x 1080 px Display with Water Drop Notch,Octa Core Processor,Exynos 1330
1,1,Samsung Galaxy A11,4.20,63,"Dual Sim, 3G, 4G, VoLTE,",2 GB RAM,4000 mAh Battery,6.4 inches,13 MP + 5 MP + 2 MP Triple Rear &amp; 8 MP Fro...,"Memory Card Supported, upto 512 GB",10,"9,990",Samsung,32 GB inbuilt,15W Fast Charging,720 x 1560 px Display with Punch Hole,1.8 GHz Processor,Octa Core
2,2,Samsung Galaxy A13,4.30,75,"Dual Sim, 3G, 4G, VoLTE,",4 GB RAM,5000 mAh Battery,6.6 inches,50 MP Quad Rear &amp; 8 MP Front Camera,"Memory Card Supported, upto 1 TB",12,"11,999",Samsung,64 GB inbuilt,25W Fast Charging,1080 x 2408 px Display with Water Drop Notch,2 GHz Processor,Octa Core
3,3,Samsung Galaxy F23,4.10,73,"Dual Sim, 3G, 4G, VoLTE,",4 GB RAM,6000 mAh Battery,6.4 inches,48 MP Quad Rear &amp; 13 MP Front Camera,"Memory Card Supported, upto 1 TB",12,"11,999",Samsung,64 GB inbuilt,NaN,720 x 1600 px,Octa Core,Helio G88
4,4,Samsung Galaxy A03s (4GB RAM + 64GB),4.10,69,"Dual Sim, 3G, 4G, VoLTE,",4 GB RAM,5000 mAh Battery,6.5 inches,13 MP + 2 MP + 2 MP Triple Rear &amp; 5 MP Fro...,"Memory Card Supported, upto 1 TB",11,"11,999",Samsung,64 GB inbuilt,15W Fast Charging,720 x 1600 px Display with Water Drop Notch,Octa Core,Helio P35


In [130]:
import pandas as pd
import numpy as np
import re  # For extracting numbers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load dataset
df = pd.read_csv("/content/train.csv")

# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0', 'Name'])  # Removing non-useful columns

# Function to extract numbers from strings
def extract_numbers(text):
    num = re.findall(r'\d+', str(text))  # Find all digits
    return int(num[0]) if num else np.nan  # Convert to int, else NaN

# Fix Price column (Remove commas and convert to int)
df['Price'] = df['Price'].apply(lambda x: int(str(x).replace(',', '')) if pd.notna(x) else np.nan)

# Apply extraction to relevant columns
df['Ram'] = df['Ram'].apply(extract_numbers)  # Extract RAM in GB
df['Battery'] = df['Battery'].apply(extract_numbers)  # Extract battery in mAh
df['Display'] = df['Display'].apply(lambda x: float(re.findall(r'[\d.]+', str(x))[0]) if re.findall(r'[\d.]+', str(x)) else np.nan)  # Extract inches
df['Camera'] = df['Camera'].apply(lambda x: extract_numbers(x.split('&')[0]))  # Extract primary camera MP
df['Android_version'] = df['Android_version'].apply(extract_numbers)  # Extract Android version (e.g., 12 from "Android v12")

# Fix External_Memory column
def extract_external_memory(text):
    num = re.findall(r'\d+', str(text))  # Extract numerical value
    return int(num[0]) if num else 0  # If no external memory, return 0

df['External_Memory'] = df['External_Memory'].apply(extract_external_memory)  # Convert to integer

# Fix Inbuilt_memory column
df['Inbuilt_memory'] = df['Inbuilt_memory'].apply(extract_numbers)  # Extract storage in GB

# Fix Screen_resolution column
def extract_screen_resolution(resolution):
    numbers = re.findall(r'\d+', str(resolution))
    if len(numbers) >= 2:
        width = int(numbers[0])
        height = int(numbers[1])
        return width * height  # Convert to total pixels
    return np.nan  # Return NaN if no valid resolution

df['Screen_resolution'] = df['Screen_resolution'].apply(extract_screen_resolution)

# Fix Processor column (Extract core count)
def extract_processor_cores(text):
    text = str(text).lower()  # Convert to lowercase
    if "octa" in text:
        return 8
    elif "hexa" in text:
        return 6
    elif "quad" in text:
        return 4
    elif "dual" in text:
        return 2
    else:
        return np.nan  # Assign NaN for unknown types

df['Processor'] = df['Processor'].apply(extract_processor_cores)

# Encode categorical variables
label_enc = LabelEncoder()
df['company'] = label_enc.fit_transform(df['company'])
df['Processor_name'] = label_enc.fit_transform(df['Processor_name'])

# Handle categorical features (One-Hot Encoding)
df = pd.get_dummies(df, columns=['fast_charging', 'No_of_sim'])

# Drop rows with missing values (if any)
df = df.dropna()

# Split data into features and target
X = df.drop(columns=['Price'])
y = df['Price']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")


Mean Absolute Error: 4134.770795681063
Root Mean Squared Error: 10239.735388988469


In [131]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)

print(f"R-squared Score: {r2 * 100:.2f}%")

R-squared Score: 75.96%


In [ ]:
def predict_price():
    print("\n🔮 Enter mobile specifications to predict price 🔮")

    ram = int(input("Enter RAM (GB): "))
    battery = int(input("Enter Battery (mAh): "))
    display = float(input("Enter Display size (in inches): "))
    camera = int(input("Enter Primary Camera (MP): "))
    android_version = int(input("Enter Android Version: "))
    external_memory = int(input("Enter External Memory (GB): "))
    inbuilt_memory = int(input("Enter Inbuilt Memory (GB): "))
    screen_res = int(input("Enter Screen Resolution (total pixels, e.g., 1080x2400 → 2592000): "))

    processor_type = input("Enter Processor Type (Octa/Hexa/Quad/Dual): ").strip().lower()
    processor = 8 if "octa" in processor_type else 6 if "hexa" in processor_type else 4 if "quad" in processor_type else 2

    company = input("Enter Company Name: ").strip().lower()
    if company in label_enc.classes_:
        company = label_enc.transform([company])[0]
    else:
        company = 0  # Default if not in dataset

    processor_name = input("Enter Processor Name: ").strip().lower()
    if processor_name in label_enc.classes_:
        processor_name = label_enc.transform([processor_name])[0]
    else:
        processor_name = 0  # Default if not in dataset

    fast_charging = int(input("Fast Charging (1 for Yes, 0 for No): "))
    no_of_sim = int(input("No of SIMs (1 for Single, 2 for Dual): "))

    # Create DataFrame for input
    user_df = pd.DataFrame([[ram, battery, display, camera, android_version,
                             external_memory, inbuilt_memory, screen_res, processor,
                             company, processor_name, fast_charging, no_of_sim]],
                           columns=['Ram', 'Battery', 'Display', 'Camera', 'Android_version',
                                    'External_Memory', 'Inbuilt_memory', 'Screen_resolution', 'Processor',
                                    'company', 'Processor_name', 'fast_charging_1', 'No_of_sim_2'])

    # Ensure all expected columns exist in user_df
    for col in X.columns:
        if col not in user_df.columns:
            user_df[col] = 0  # Assign 0 to missing one-hot encoded columns

    # Reorder columns to match training data
    user_df = user_df[X.columns]

    # Scale input
    user_scaled = scaler.transform(user_df)

    # Predict price
    predicted_price = model.predict(user_scaled)
    print(f"\n💰 Predicted Price: ₹{int(predicted_price[0])}")

predict_price()



🔮 Enter mobile specifications to predict price 🔮
